In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


25/02/28 07:53:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [20]:
spark.sql("SHOW DATABASES IN demo").show()


+---------+
|namespace|
+---------+
|      nyc|
+---------+



In [3]:
# Get list of tables in the 'nyc' database
tables = spark.sql("SHOW TABLES IN nyc")
tables.show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|      nyc|           taxis_100|      false|
|      nyc|     taxis_100M_time|      false|
|      nyc|taxis_1000_50COLUMNS|      false|
|      nyc|            taxis_1B|      false|
|      nyc|taxis_100_50COLUM...|      false|
|      nyc|taxis_100000_50CO...|      false|
|      nyc|            taxis_1L|      false|
|      nyc|          taxis_1000|      false|
|      nyc|            taxis_10|      false|
|      nyc|   taxis_1L_5COLUMNS|      false|
|      nyc|         taxis_10000|      false|
|      nyc|            taxis_1M|      false|
|      nyc|          taxis_1L_5|      false|
|      nyc|          taxis_10_M|      false|
|      nyc|taxis_1000_50COLU...|      false|
|      nyc|  taxis_10_50COLUMNS|      false|
|      nyc|           taxis_10K|      false|
|      nyc|            taxis_1K|      false|
|      nyc|           taxis_10L|      false|
|      nyc

In [25]:
spark.conf.set("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.demo.type", "hadoop")  # or "hadoop" for local file system


In [26]:
df=spark.sql("DESCRIBE demo.nyc.taxis_10000_50COLUMNS")
df.show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
| extra_col_0|   string|   NULL|
| extra_col_1|      int|   NULL|
| extra_col_2|   string|   NULL|
| extra_col_3|     date|   NULL|
| extra_col_4|   string|   NULL|
| extra_col_5|      int|   NULL|
| extra_col_6|   string|   NULL|
| extra_col_7|     date|   NULL|
| extra_col_8|   string|   NULL|
| extra_col_9|      int|   NULL|
|extra_col_10|   string|   NULL|
|extra_col_11|     date|   NULL|
|extra_col_12|   string|   NULL|
|extra_col_13|      int|   NULL|
|extra_col_14|   string|   NULL|
|extra_col_15|     date|   NULL|
|extra_col_16|   string|   NULL|
|extra_col_17|      int|   NULL|
|extra_col_18|   string|   NULL|
|extra_col_19|     date|   NULL|
+------------+---------+-------+
only showing top 20 rows



In [30]:
df=spark.sql("SELECT * FROM demo.nyc.taxis_10000_50COLUMNS.history")
df.show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-02-27 06:59:...|7430131516402498764|               NULL|               true|
|2025-02-27 07:00:...|2768008278032157245|7430131516402498764|               true|
|2025-02-27 07:01:...|5954078949605399424|2768008278032157245|               true|
+--------------------+-------------------+-------------------+-------------------+



In [31]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_10000_50COLUMNS.history ORDER BY made_current_at DESC")

df.show()

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
|5954078949605399424|2025-02-27 07:01:...|
|2768008278032157245|2025-02-27 07:00:...|
|7430131516402498764|2025-02-27 06:59:...|
+-------------------+--------------------+



In [39]:
df=spark.sql("SELECT * FROM demo.nyc.taxis_10000_50COLUMNS.snapshots")
df.show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-02-27 06:59:...|7430131516402498764|               NULL|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-02-27 07:00:...|2768008278032157245|7430131516402498764|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-02-27 07:01:...|5954078949605399424|2768008278032157245|overwrite|s3://warehouse/ny...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



In [43]:
df = spark.sql("SELECT * FROM demo.nyc.taxis_10000_50COLUMNS FOR SYSTEM_VERSION AS OF 7430131516402498764")
df.show()


+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

25/02/28 08:47:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [44]:
df = spark.sql("SELECT * FROM demo.nyc.taxis_10000_50COLUMNS FOR SYSTEM_VERSION AS OF 2768008278032157245")
df.show()

+-----------+-----------+---------------+-----------+-----------+-----------+---------------+-----------+-----------+-----------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+
|extra_col_0|extra_col_1|    extra_col_2|extra_col_3|extra_col_4|extra_col_5|    extra_col_6|extra_col_7|extra_col_8|extra_col_9|   extra_col_10|extra_col_11|extra_col_12|extra_col_13|   extra_col_14|extra_col_15|extra_col_16|extra_col_17|   extra_col_18|extra_col_19|extra_col_20|extra_col_21|   extra_col_22|extra_col_

In [45]:
df = spark.sql("SELECT * FROM demo.nyc.taxis_10000_50COLUMNS FOR SYSTEM_VERSION AS OF 5954078949605399424")
df.show()

+-----------+-----------+---------------+-----------+-----------+-----------+---------------+-----------+-----------+-----------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+---------------+------------+------------+------------+
|extra_col_0|extra_col_1|    extra_col_2|extra_col_3|extra_col_4|extra_col_5|    extra_col_6|extra_col_7|extra_col_8|extra_col_9|   extra_col_10|extra_col_11|extra_col_12|extra_col_13|   extra_col_14|extra_col_15|extra_col_16|extra_col_17|   extra_col_18|extra_col_19|extra_col_20|extra_col_21|   extra_col_22|extra_col_

In [47]:
df = spark.sql("""
SELECT 
    t1.extra_col_0, 
    t1.extra_col_1 AS extra_col_1_version_1, 
    t2.extra_col_1 AS extra_col_1_version_2
FROM 
    demo.nyc.taxis_10000_50COLUMNS FOR SYSTEM_VERSION AS OF 2768008278032157245 t1
JOIN 
    demo.nyc.taxis_10000_50COLUMNS FOR SYSTEM_VERSION AS OF 5954078949605399424 t2
ON 
    t1.extra_col_0 = t2.extra_col_0
WHERE 
    t1.extra_col_1 != t2.extra_col_1
""")
df.show()


+-----------+---------------------+---------------------+
|extra_col_0|extra_col_1_version_1|extra_col_1_version_2|
+-----------+---------------------+---------------------+
| ESFQDGITAC|                 4245|                 4255|
| GHCTPFZNLI|                 6931|                 6941|
| YGLRSOETYW|                 4121|                 4131|
| XYGCFDSEDM|                 7519|                 7529|
| FHLWFPWQKX|                 9012|                 9022|
| ESSXORMSXT|                 4580|                 4590|
| KQAIBHIHGM|                 6835|                 6845|
| OVSTTFJHTM|                   43|                   53|
| NHPBPGTHNU|                 2231|                 2241|
| QLSKCGZDGW|                  476|                  486|
| ZMVNSOMSNW|                 8268|                 8278|
| KEIGVIHNCJ|                 2858|                 2868|
| DRBEYKPRRW|                 1188|                 1198|
| MPVZTEDJYR|                 1774|                 1784|
| JLLOFWKIMO| 